## Import Libraries

In [1]:
import pandas as pd # type: ignore
import plotly.express as px # type: ignore

import plotly.io as pio

## Load Data

In [2]:
# Load the dataset
df = pd.read_csv('../vehicles_us.csv')

## Exploratory Data Analysis

In [3]:
# Basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [4]:
# Preview data
df.head()


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [5]:
# Shape of Data
print("Shape of the dataset:", df.shape)

Shape of the dataset: (51525, 13)


In [6]:
# Summary statistics
df.describe()

,price,model_year,cylinders,odometer,is_4wd,days_listed
count,51525.000000,47906.000000,46265.000000,43633.000000,25572.0,51525.00000
mean,12132.464920,2009.750470,6.125235,115553.461738,1.0,39.55476
std,10040.803015,6.282065,1.660360,65094.611341,0.0,28.20427
min,1.000000,1908.000000,3.000000,0.000000,1.0,0.00000
25%,5000.000000,2006.000000,4.000000,70000.000000,1.0,19.00000
50%,9000.000000,2011.000000,6.000000,113000.000000,1.0,33.00000
75%,16839.000000,2014.000000,8.000000,155000.000000,1.0,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,1.0,271.00000


In [7]:
# Checking for missing values
df.isnull().sum()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

In [8]:
# List of columns in the dataset
print(df.columns)

Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed'],
      dtype='object')


In [9]:
# Clean column names
df.columns = df.columns.str.strip().str.lower()
df['model_year'] = pd.to_numeric(df['model_year'], errors='coerce')
df['cylinders'] = pd.to_numeric(df['cylinders'], errors='coerce')
df['odometer'] = pd.to_numeric(df['odometer'], errors='coerce')

In [10]:
# ====== Fill missing values using group strategies ======

# Fill model_year: median by model
df['model_year'] = df.groupby('model')['model_year'].transform(lambda x: x.fillna(x.median()))

# Fill cylinders: median by model
df['cylinders'] = df.groupby('model')['cylinders'].transform(lambda x: x.fillna(x.median()))

# Fill odometer: median by model + model_year
df['odometer'] = df.groupby(['model', 'model_year'])['odometer'].transform(lambda x: x.fillna(x.median()))


In [11]:
# ====== Fill remaining missing values with defaults ======
df['is_4wd'] = df['is_4wd'].fillna(0).astype(int)
df['paint_color'] = df['paint_color'].fillna("unknown")
df['condition'] = df['condition'].fillna("unknown")
df['transmission'] = df['transmission'].fillna("unknown")
df['fuel'] = df['fuel'].fillna("unknown")
df['type'] = df['type'].fillna("unknown")


In [12]:
# ====== Create new columns ======
df['model_year'] = df['model_year'].astype(int)
df['age'] = 2023 - df['model_year']

In [13]:
# ====== Remove outliers ======
df = df[(df['price'] > 100) & (df['price'] < 100000)]
df = df[(df['model_year'] > 1980) & (df['model_year'] < 2024)]


## Visualizations

In [14]:
pio.renderers.default = 'browser'  # This opens plots in web browser

In [15]:
# 1. Age vs Price Scatter
fig1 = px.scatter(df, x='age', y='price', title='Car Age vs Price')
fig1.show()

In [16]:
# 2. Price Distribution
fig2 = px.histogram(df, x='price', nbins=50, title='Price Distribution')
fig2.show()

In [17]:
# 3. Average Price by Condition
fig3 = px.bar(df.groupby('condition')['price'].mean().reset_index(),
              x='condition', y='price', title='Average Price by Condition')
fig3.show()

In [18]:
# 4. Average Odometer by Model Year
fig4 = px.line(df.groupby('model_year')['odometer'].mean().reset_index(),
               x='model_year', y='odometer', title='Average Odometer by Year')
fig4.show()

In [19]:
# 5. Scatter plot for Mileage vs Price
fig = px.scatter(df, x='odometer', y='price', title='Odometer (Mileage) vs Price')
fig.show()

In [21]:
# 6. Average Price by Model
n = 20 
top_models = df['model'].value_counts().head(20).index
top_df = df[df['model'].isin(top_models)]
avg_price_by_model = top_df.groupby('model')['price'].mean().reset_index()

fig = px.bar(avg_price_by_model, x='model', y='price', title=f"Average Price (Top {n} Models)")
fig.show()

## Save cleaned dataset

In [22]:
# ====== Save cleaned data for app.py ======
df.to_csv("../vehicles_cleaned.csv", index=False)

In [22]:
for col in df.columns:
    print(col, df[col].apply(type).unique())


price [<class 'int'>]
model_year [<class 'int'>]
model [<class 'str'>]
condition [<class 'str'>]
cylinders [<class 'float'>]
fuel [<class 'str'>]
odometer [<class 'float'>]
transmission [<class 'str'>]
type [<class 'str'>]
paint_color [<class 'str'>]
is_4wd [<class 'int'>]
date_posted [<class 'str'>]
days_listed [<class 'int'>]
age [<class 'int'>]
